## Import des bibliothèques

In [44]:
import sqlite3
from faker import Faker
import random
from datetime import datetime, timedelta

## Creation d'une BDD originale

In [77]:


fake = Faker('fr_FR')

# /Création/Connexion à la base de données SQLite
connexion = sqlite3.connect('PassionPaddle5.db')
cursor = connexion.cursor()

# Création de la table des employés
cursor.execute('''CREATE TABLE employes (
                    id_em INTEGER PRIMARY KEY,
                    nom TEXT,
                    genre TEXT,
                    date_naissance DATE,
                    date_embauche DATE,
                    date_depart DATE,
                    id_li INTEGER,
                    FOREIGN KEY (id_li) REFERENCES lieux (id_li)
                )''')

# Création de la table des lieux de travail
cursor.execute('''CREATE TABLE lieux (
                    id_li INTEGER PRIMARY KEY,
                    ville TEXT
                )''')

# Génération des données pour la table des employés
for _ in range(20):
    nom = fake.last_name()
    genre = random.choice(['Homme', 'Femme'])
    date_naissance = fake.date_of_birth(minimum_age=20, maximum_age=63)

    yesterday = datetime.now().date() - timedelta(days=1)
    date_embauche = fake.date_between(start_date=date_naissance + timedelta(days=365*18), end_date=yesterday)
    
    date_depart = random.choice(['9999-01-01', None])
    if date_depart != '9999-01-01':
        # date_embauche_dt = datetime.strptime(date_embauche)
        date_depart_dt = fake.date_between_dates(date_start=date_embauche + timedelta(days=1),
                                                 date_end=datetime.today())
        date_depart = date_depart_dt
    
    id_li = _ + 1  # id_li est égal à id_em associé
    

    # Insertion des données dans la table des employés
    cursor.execute('''INSERT INTO employes (nom, genre, date_naissance, date_embauche, date_depart, id_li)
                      VALUES (?, ?, ?, ?, ?, ?)''', (nom, genre, date_naissance, date_embauche, date_depart, id_li))

# Génération des données pour la table des lieux de travail
for ville in range(20):
    ville = random.choice(['Orléans', 'Blois', 'Tours', 'Bourges'])    # Je peux sans doute remonter tout ça dans la boucle du dessus
    # Insertion des données dans la table des lieux de travail
    cursor.execute('''INSERT INTO lieux (ville)
                      VALUES (?)''', (ville,))

# Validation et fermeture de la connexion à la base de données
connexion.commit()
connexion.close()


## Saisie des données par un utilisateur

In [32]:

# fonction utilisée pour s'assurer que les dates seront au format international pour les traiter ensuite en SQL

def validate_date(date_string):
    try:
        date = datetime.strptime(date_string, "%Y-%m-%d")
        return date.date()
    except ValueError:
        return None

# la fonction de saisie des informations par l'utilisateur

def inserer_informations_nouvel_employe() :
   
    genre_possible = ["Homme", "Femme"]
    ville_possible = ['Orléans', 'Blois', 'Tours', 'Bourges']

    nom = input("Quel est le nom de l'employé: ") 

    genre = None
    while genre not in genre_possible :
        genre = input("Quelle est le genre de l'employé (Homme/Femme) : ")
        if genre and genre not in genre_possible:
            print("La valeur saisie n'est pas correcte. Veuillez réessayer.")

    date_naissance = None
    while date_naissance is None:
        date_naissance = validate_date(input("Date de naissance (Format: yyyy-mm-dd) : "))
        if date_naissance is None:
            print("La date saisie est invalide. Veuillez réessayer.")
    
    # Validation de la date d'embauche
    date_embauche = None
    while date_embauche is None:
        date_embauche = validate_date(input("Date d'embauche (Format: yyyy-mm-dd) : "))
        if date_embauche is None:
            print("La date saisie est invalide. Veuillez réessayer.")
        else:
            age_min_requis = date_naissance + timedelta(days=16*365)  # Calcul de la date de naissance + 16 ans
            if date_embauche < age_min_requis:
                print("La date saisie est invalide. Veuillez réessayer.")
    
    # Validation de la date de départ
    date_depart = None
    while date_depart is None:
        date_depart_str = input("Date de départ (Format: yyyy-mm-dd ou 9999-01-01 pour aucune date de départ) : ")
        if date_depart_str == '9999-01-01':
            date_depart = date_depart_str
        else:
            date_depart = validate_date(date_depart_str)
            if date_depart is None or date_depart <= date_embauche:
                print("La date saisie est invalide. Veuillez réessayer.")

    #  Validation de la ville 
    ville = None           
    while ville not in ville_possible :
        ville = input("Ville d'affectation de l'employé (Orléans/Blois/Tours/Bourges) : ")
        if ville and ville not in ville_possible:
            print("La valeur saisie n'est pas correcte. Veuillez réessayer.")

    employe_data = [nom, genre, date_naissance, date_embauche, date_depart]
    ville_data =[ville]
    
    return employe_data, ville_data

#Connexion à la base de données

connexion = sqlite3.connect('PassionPaddle.db')
cursor = connexion.cursor()

encore_une_saisie = "o"


while encore_une_saisie.lower() != "n":
    employe_data,ville_data = inserer_informations_nouvel_employe()
    print(employe_data)
    
    # Insérer les données dans la base de données ici

    
    try : 
        cursor.execute("INSERT INTO lieux (ville) VALUES(?)", ville_data)
        cursor.execute("INSERT INTO employes (nom, genre, date_naissance, date_embauche, date_depart, id_li) VALUES (?, ?, ?, ?, ?, last_insert_rowid())", employe_data)
    except sqlite3.IntegrityError as e:
        print(e)
    
    encore_une_saisie = input("Voulez-vous continuer (o/n) : ")

   

['Texier', 'Homme', datetime.date(1968, 12, 7), datetime.date(2020, 2, 14), datetime.date(2021, 10, 29)]


 ## Validation et fermeture de la connexion à la base de données

In [92]:

connexion.commit()
connexion.close()    

ProgrammingError: Cannot operate on a closed database.

## Interrogation de la base

In [78]:
connexion = sqlite3.connect('PassionPaddle5.db')
cursor = connexion.cursor()

In [52]:
# Combien de salariés comprend la société ?

cursor.execute("SELECT COUNT(*) FROM employes")
total_salaries = cursor.fetchone()[0]
print(f"Le nombre total de salariés est de :{total_salaries}")



Le nombre total de salariés est de : 20


In [79]:
#Où sont-ils localisés au niveau professionnel ?
cursor.execute("SELECT ville, COUNT(*) FROM lieux JOIN employes ON lieux.id_li = employes.id_li GROUP BY ville")
localisation_professionnelle = cursor.fetchall()
print("Localisation professionnelle des salariés :")
for row in localisation_professionnelle:
    ville, count = row
    print(f"{ville}: {count} salariés")

Localisation professionnelle des salariés :
Blois: 6 salariés
Bourges: 2 salariés
Orléans: 5 salariés
Tours: 7 salariés


In [80]:
# Quel est l’âge moyen et médian des salariés ?

cursor.execute("SELECT AVG(strftime('%Y', 'now') - strftime('%Y', date_naissance)) FROM employes")
age_moyen = cursor.fetchone()[0]
print(f"Âge moyen des salariés : {age_moyen} ans.")

#Stackoverflow : requete permettant de definir la mediane que le nombre d'enregistrement soit pair ou impair

cursor.execute("""
    SELECT strftime('%Y', 'now') - strftime('%Y', date_naissance) AS age
    FROM employes
    ORDER BY age
    LIMIT 2 - (SELECT COUNT(*) FROM employes) % 2 
    OFFSET (SELECT (COUNT(*) - 1) / 2
        FROM employes
    )
""")
age_median = cursor.fetchone()[0]
print(f"Âge médian des salariés : {age_median} ans.")

Âge moyen des salariés : 39.5 ans.
Âge médian des salariés : 36 ans.


In [81]:
#Quel est l’étendue des âges ?

cursor.execute("SELECT MAX(strftime('%Y', 'now') - strftime('%Y', date_naissance)), MIN(strftime('%Y', 'now') - strftime('%Y', date_naissance)) FROM employes")
age_max, age_min = cursor.fetchone()
print(f"Âge maximum des salariés : {age_max} ans.")
print(f"Âge minimum des salariés : {age_min} ans.")

Âge maximum des salariés : 59 ans.
Âge minimum des salariés : 21 ans.


In [84]:
# Quel est le salarié le plus ancien de la société en termes d’ancienneté ?

cursor.execute("SELECT id_em,nom FROM employes WHERE date_depart = '9999-01-01' ORDER BY date_embauche ASC LIMIT 1")
salarie_plus_ancien = cursor.fetchone()
print(f"Le salarié le plus ancien est {salarie_plus_ancien[1]}, Id n° {salarie_plus_ancien[0]}:")

Le salarié le plus ancien est Lemaître, Id n° 14:


In [85]:
# Quelle est la répartition entre les hommes et les femmes ?

cursor.execute("""
    SELECT
        (COUNT(CASE WHEN genre = 'Homme' THEN 1 ELSE NULL END) * 100 / COUNT(*)) AS pourcentage_hommes,
        (COUNT(CASE WHEN genre = 'Femme' THEN 1 ELSE NULL END) * 100 / COUNT(*)) AS pourcentage_femmes
    FROM employes
""")

result = cursor.fetchone()

#  Pourcentages d'hommes et de femmes
pourcentage_hommes = result[0]
pourcentage_femmes = result[1]

print(f"Répartition hommes-femmes :")
print(f"Pourcentage d'hommes : {pourcentage_hommes:}%")
print(f"Pourcentage de femmes : {pourcentage_femmes:}%")


Répartition hommes-femmes :
Pourcentage d'hommes : 50%
Pourcentage de femmes : 50%


In [86]:
#Y-a-t-il des doublons ?

cursor.execute("SELECT id_em, COUNT(*) FROM employes GROUP BY nom, genre, date_naissance HAVING COUNT(*) > 1")
doublons = cursor.fetchall()
if doublons:
    print("Doublons détectés :")
    for doublon in doublons:
        print(f"Numéro d'employé en doublon : {doublon[0]}")
else:
    print("Aucun doublon trouvé.")


Aucun doublon trouvé.


In [87]:
# Afficher la liste des salariés classés par ordre alphabétique.

cursor.execute("SELECT nom FROM employes ORDER BY nom")
liste_salaries = cursor.fetchall()
print("Liste des salariés par ordre alphabétique :")
for row in liste_salaries:
    print(row[0])


Liste des salariés par ordre alphabétique :
Barbier
Bertrand
Brunel
Bègue
Collet
Fernandes
Grondin
Lambert
Legendre
Lejeune
Lemaître
Martin
Mercier
Neveu
Pasquier
Ruiz
Schmitt
Teixeira
Teixeira
Thierry


In [62]:
# Fermeture de la bdd

connexion.close() 